In [ ]:
!pip install -q transformers
!pip install -q datasets
     

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 13.9 MB/s eta 0:00:00


##Reading files

In [ ]:
import os
import numpy as np
import pandas as pd

data_folder = "/content/drive/MyDrive/Emotions_data"
dataset = []

# Iterate over the files in the data folder
for filename in os.listdir(data_folder):
    # Extract the emotion label from the file name
    emotion_label = filename[:4]  # Assuming the first 4 characters are the label
    
    # Read in the text of the poem
    with open(os.path.join(data_folder, filename), "r") as f:
        poem_text = f.read()
        
    # Add the poem and emotion label to the dataset
    dataset.append((poem_text, emotion_label))

import chardet

with open("/content/all.csv", 'rb') as f:
    result = chardet.detect(f.read())

new_df = pd.read_csv("/content/all.csv", encoding=result['encoding'])

# Extract the content and type columns as lists
text = new_df["content"].tolist()
label = new_df["type"].tolist()

# Append the text and label lists to the dataset list
for i in range(len(text)):
    dataset.append((text[i], label[i]))

print(np.size(dataset))


2364


##Exploring data

In [ ]:

df = pd.DataFrame(dataset, columns=["poem_text", "emotion_label"])
df.head

<bound method NDFrame.head of                                               poem_text emotion_label
0     Anger is a virus\nThat needs not even air\nTo ...          Ange
1     As Cloe came into the Room t'other Day,\nI pee...          Ange
2     Any time I don't understand something\nOr feel...          Ange
3     When I am full of anger\nI think in anger's te...          Ange
4     My anger is angry.\nIts pain.\nIts frustration...          Ange
...                                                 ...           ...
1177  Daughter Taken By Mothers Lies\n\nHave you any...           sad
1178  Involuntary Acceptance\n\nEven though\nWe’re f...           sad
1179  Victim Of Poverty\n\nPoverty stricken youth ju...           sad
1180  Rain\n\nI sit and watch\nas the rain falls \nf...           sad
1181  The Face Of Sadness\n\nIts happened again.\n\n...           sad

[1182 rows x 2 columns]>

In [ ]:
print(df.emotion_label.unique())

['Ange' 'Fear' 'JoyP' 'Love' 'Sorr' 'Anger' 'fear' 'joy' 'sad']


##Converting the labels to integers

In [ ]:
emotion_to_int = {"Fear": 0, "Ange": 1, "JoyP": 2, "Love": 3, "Sorr": 4, "sad":4,"anger":1,"fear":0,"joy":2,"Anger":1}  # Map each emotion to an integer
# Convert the emotion labels to integers
df["emotion_label"] = df["emotion_label"].apply(lambda x: emotion_to_int[x])


In [ ]:
df.head

<bound method NDFrame.head of                                               poem_text  emotion_label
0     Anger is a virus\nThat needs not even air\nTo ...              1
1     As Cloe came into the Room t'other Day,\nI pee...              1
2     Any time I don't understand something\nOr feel...              1
3     When I am full of anger\nI think in anger's te...              1
4     My anger is angry.\nIts pain.\nIts frustration...              1
...                                                 ...            ...
1177  Daughter Taken By Mothers Lies\n\nHave you any...              4
1178  Involuntary Acceptance\n\nEven though\nWe’re f...              4
1179  Victim Of Poverty\n\nPoverty stricken youth ju...              4
1180  Rain\n\nI sit and watch\nas the rain falls \nf...              4
1181  The Face Of Sadness\n\nIts happened again.\n\n...              4

[1182 rows x 2 columns]>

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

##Loading the pre-trained BERT model for fine tuning

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=5)


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW

In [ ]:
# Define a custom dataset class for annotated poems
class PoemDataset(Dataset):
    def __init__(self, data, tokenizer):
        self.tokenizer = tokenizer
        self.texts = data['poem_text'].tolist()
        self.labels = data['emotion_label'].tolist()
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, truncation=True, padding='max_length', max_length=128, return_tensors='pt')
        return {'input_ids': encoding['input_ids'][0], 'attention_mask': encoding['attention_mask'][0], 'label': label}

In [ ]:
from sklearn.model_selection import train_test_split

train_data, val_data = train_test_split(df, test_size=0.2, random_state=72)


train_dataset = PoemDataset(train_data, tokenizer)
val_dataset = PoemDataset(val_data, tokenizer)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [ ]:
train_data

,poem_text,emotion_label
488,"By the river,\nunder the sorrow tree,\nthe uni...",4
470,My soul is cold as ice.\nMy heart is darker th...,4
1030,The Blue Bird\n\n\nI am the bird of God in His...,4
638,"Now thou has loved me one whole day,\r\nTomorr...",3
999,Cop Contest\n\nA dispute among three kinds of ...,2
...,...,...
1031,A God's Labour\n\nI have gathered my dreams in...,4
837,I Don't Understand\nHow are you Daddy?\nIt's b...,1
1098,My Friend \n\nArt thou abroad on this stormy n...,4
1070,Hymns of a hag\n\nI fancy myself being a witch...,4


In [ ]:
val_data

,poem_text,emotion_label
259,Perfume of tenderness\nSoft hugs of kindness\n...,2
362,he offered her all the treasures that she desi...,3
474,"My dear friend, but for clouds in your eyes;\n...",4
803,Version 1 (1921)\r\nYours is the shame and sor...,3
36,people can be so stupid\nbut think they are al...,1
...,...,...
495,The lethal bullets of sorrow\nAim at our rosy ...,4
267,My heart is like a little bird\nThat sits and ...,2
479,Why does the thin grey strand\nFloating up fro...,4
539,"All Kings, and all their favourites,\r\n ...",3


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

##Defining and Training the model and fine-tuning 

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 16
logging_steps = len(train_data) // batch_size
training_args = TrainingArguments(output_dir="results",
                                  num_train_epochs=10,
                                  learning_rate=0.0001,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  load_best_model_at_end=True,
                                  metric_for_best_model="f1",
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  save_strategy="epoch",
                                  disable_tqdm=False)

In [ ]:
from transformers import Trainer
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train();

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.923487,0.654008,0.665269
2,No log,0.706279,0.759494,0.773746
3,No log,0.712194,0.767932,0.772698
4,No log,1.042397,0.772152,0.773855
5,No log,1.133580,0.755274,0.765677
6,No log,1.228170,0.755274,0.769070
7,No log,1.076184,0.793249,0.793303
8,No log,1.097399,0.784810,0.787601
9,0.327000,1.085652,0.801688,0.802858
10,0.327000,1.088275,0.793249,0.793928


In [ ]:
results = trainer.evaluate()
results
     

{'eval_loss': 1.085652232170105,
 'eval_accuracy': 0.8016877637130801,
 'eval_f1': 0.8028577294436747,
 'eval_runtime': 1.9208,
 'eval_samples_per_second': 123.387,
 'eval_steps_per_second': 7.809,
 'epoch': 10.0}

In [ ]:
model.save_pretrained('./model')
tokenizer.save_pretrained('./model')

('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

##Testing the model on a random poem

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("/content/model")
model = AutoModelForSequenceClassification.from_pretrained("/content/model")

# Set the model to evaluation mode
model.eval()

# Read text from a file
with open('/content/LustPoemsiMInLustPoembyEffieYalenaSteyn.txt', 'r') as f:
    text = f.read()

# Tokenize the text and truncate or pad it to the desired length
inputs = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')

# Convert tokenized input to tensors
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

# Pass the input tensors through the model and get the predicted output probabilities
outputs = model(input_ids, attention_mask=attention_mask)
probs = torch.softmax(outputs.logits, dim=-1)

print(outputs)
print(probs)
# Extract the highest probability class index
preds = torch.argmax(probs, dim=-1)
print(preds)
print(preds.item())




SequenceClassifierOutput(loss=None, logits=tensor([[-2.5586, -1.8955, -2.0049,  7.0514, -0.2427]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[6.6992e-05, 1.3002e-04, 1.1655e-04, 9.9901e-01, 6.7891e-04]],
       grad_fn=<SoftmaxBackward0>)
tensor([3])
3


##Testing another sample

In [ ]:
# Read text from a file
with open('/content/AlonePoemsIAmMuchTooAloneInThisWorldYetNotAlonePoembyRainerMariaRilke.txt', 'r') as f:
    text = f.read()

# Tokenize the text and truncate or pad it to the desired length
inputs1 = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')

# Convert tokenized input to tensors
input_ids1 = inputs1['input_ids']
attention_mask1 = inputs1['attention_mask']

# Pass the input tensors through the model and get the predicted output probabilities
outputs1 = model(input_ids1, attention_mask=attention_mask1)
probs1 = torch.softmax(outputs1.logits, dim=-1)

print(outputs1)
print(probs1)
# Extract the highest probability class index
preds1 = torch.argmax(probs1, dim=-1)
print(preds1)
print(preds1.item())

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.4211, -0.9410, -2.2461, -2.1655,  2.7704]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[0.4045, 0.0140, 0.0038, 0.0041, 0.5736]], grad_fn=<SoftmaxBackward0>)
tensor([4])
4
